In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
def calculate_technical_indicators(df):
    # RSI
    def calculate_rsi(data, periods=14):
        delta = data.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))
    
    # MACD
    def calculate_macd(data, fast=12, slow=26, signal=9):
        exp1 = data.ewm(span=fast, adjust=False).mean()
        exp2 = data.ewm(span=slow, adjust=False).mean()
        macd = exp1 - exp2
        signal_line = macd.ewm(span=signal, adjust=False).mean()
        return macd, signal_line
    
    # Bollinger Bands
    def calculate_bollinger_bands(data, window=20):
        ma = data.rolling(window=window).mean()
        std = data.rolling(window=window).std()
        upper = ma + (std * 2)
        lower = ma - (std * 2)
        return upper, ma, lower
    
    # Calculate indicators
    df['RSI'] = calculate_rsi(df['Close'])
    df['MACD'], df['MACD_Signal'] = calculate_macd(df['Close'])
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = calculate_bollinger_bands(df['Close'])
    
    # Volume indicators
    df['Volume_SMA'] = df['Volume'].rolling(window=20).mean()
    df['Volume_Ratio'] = df['Volume'] / df['Volume_SMA']
    
    # Price momentum
    for window in [5, 10, 20, 50, 100, 200]:
        # Price moving averages
        df[f'SMA_{window}'] = df['Close'].rolling(window=window).mean()
        df[f'Price_to_SMA_{window}'] = df['Close'] / df[f'SMA_{window}']
        
        # Returns over different periods
        df[f'Return_{window}'] = df['Close'].pct_change(periods=window)
        
        # Volatility
        df[f'Volatility_{window}'] = df['Close'].rolling(window=window).std()
        
        # High/Low range
        df[f'HL_Range_{window}'] = (
            (df['High'].rolling(window=window).max() - df['Low'].rolling(window=window).min()) /
            df['Close']
        )
    
    # Additional features
    df['Daily_Return'] = df['Close'].pct_change()
    df['Gap'] = (df['Open'] - df['Close'].shift(1)) / df['Close'].shift(1)
    df['ATR'] = (
        df['High'].rolling(14).max() - df['Low'].rolling(14).min()
    ) / df['Close'].rolling(14).mean()
    
    return df

def create_target(df, threshold=0.015):  # 1.5% threshold for stronger signals
    df['Tomorrow'] = df['Close'].shift(-1)
    df['Return'] = (df['Tomorrow'] - df['Close']) / df['Close']
    df['Target'] = (df['Return'] > threshold).astype(int)
    return df

def add_calendar_features(df):
    df['Day_of_Week'] = df.index.dayofweek
    df['Month'] = df.index.month
    df['Quarter'] = df.index.quarter
    df['Year'] = df.index.year
    
    # Add cyclical encoding for temporal features
    for col in ['Day_of_Week', 'Month']:
        df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
        df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    
    return df

def predict(train, test, predictors, model, probability_threshold=0.65):
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train[predictors])
    test_scaled = scaler.transform(test[predictors])
    
    model.fit(train_scaled, train['Target'])
    probas = model.predict_proba(test_scaled)
    preds = (probas[:, 1] > probability_threshold).astype(int)
    
    return pd.Series(preds, index=test.index, name='Predictions')

def backtest(data, model, predictors, start=252, step=30, prob_threshold=0.65):
    all_predictions = []
    
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model, prob_threshold)
        all_predictions.append(pd.DataFrame(predictions))
    
    return pd.concat(all_predictions)

# Fetch and prepare data
aapl = yf.Ticker("AAPL")
aapl = aapl.history(period="max")

# Create features
aapl = calculate_technical_indicators(aapl)
aapl = create_target(aapl)
aapl = add_calendar_features(aapl)
aapl = aapl.loc['2017-01-01':].copy()

# Drop rows with NaN values
aapl = aapl.dropna()

# Define predictors (exclude non-feature columns)
exclude_columns = ['Target', 'Tomorrow', 'Return', 'Open', 'High', 'Low', 'Close', 'Volume']
predictors = [col for col in aapl.columns if col not in exclude_columns]

# Create optimized Random Forest model
model = RandomForestClassifier(
    n_estimators=1000,  # Increased number of trees
    min_samples_split=30,
    min_samples_leaf=15,
    max_depth=8,
    max_features='sqrt',
    class_weight={0: 1, 1: 2},  # Give more weight to positive class
    random_state=42,
    n_jobs=-1  # Use all available cores
)

# Run backtest with stricter probability threshold
predictions = backtest(aapl, model, predictors, prob_threshold=0.65)

# Calculate precision score
final_precision = precision_score(aapl.iloc[-len(predictions):]['Target'], predictions['Predictions'])
print(f"Final Precision Score: {final_precision}")

# Print feature importances
feature_importance = pd.DataFrame({
    'feature': predictors,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)
print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

Final Precision Score: 0.5

Top 10 Most Important Features:
            feature  importance
29      HL_Range_50    0.045438
19      HL_Range_10    0.040371
24      HL_Range_20    0.038957
11   Price_to_SMA_5    0.035564
9      Volume_Ratio    0.030804
42              ATR    0.030627
41              Gap    0.027653
34     HL_Range_100    0.027426
8        Volume_SMA    0.027366
21  Price_to_SMA_20    0.027170
